# Joining Tables

## Cartesian Product

So far we have restricted ourselves to operators that operate on one table (relation) at a time. This is logical in the sense that our operators create relations! However, we know that a typical database contains many tables, which in fact may be related. So, how do we do queries using mulitple tables?

The first step toward applying the operators we have learned so far to multiple tables is to merge the tables together. We do this using the cartesian product.

A cartesian product creates one table out of two tables by creating every possible combination of each row in table A with each row in table B, forming a new relation with A+B columns, and A\*B rows!


### Relation 1

COL_1 | COL_2
------|------
 A | B
 C | D
 E | F
 

### Relation 2

COL_3 | COL_4
------|-------
 1 | A
 2 | C
 3 | A
 

### Cartesian Product: Result

COL_1 | COL_2 | COL_3 | COL_4
------|-------|-------|------
 A | B | 1 | A
 A | B | 2 | C
 A | B | 3 | A
 C | D | 1 | A
 C | D | 2 | C
 C | D | 3 | A
 E | F | 1 | A
 E | F | 2 | C
 E | F | 3 | A
 

### Cartesian Product: Challenge

Of course this can create an **enormous** table, so the cartesian product is always followed by a query where we limit the number of rows by comparing a column in relation 1 against a column in relation 2.


### Cartesian Product: Query

COL_1 == COL_4

COL_1 | COL_2 | COL_3 | COL_4
------|-------|-------|------
 A | B | 1 | A
 A | B | 3 | A
 C | D | 2 | C


## Natural Join

The natural join or `njoin` operator takes the pattern of cartesian product followed by query, and wraps it all into one operation subject to the following:

* The query condition tests for equality
* The query condition of equality applies to all columns with the same name in both relations

You can see this in the following diagram, where we have two relations. Both have a column named C1.

The resulting relation has a single C1 column where only the rows where C1 holds the same value in both relations. The other values from the row are filled in with the values from the matching rows.


In [1]:
import warnings
warnings.filterwarnings('ignore')

from reframe import Relation
r1 = Relation('/home/faculty/yasiro01/pub/R1.csv',sep=',')
r2 = Relation('/home/faculty/yasiro01/pub/R2.csv',sep=',')


In [2]:
r1

,C1,C2
0,1,A
1,2,B
2,3,C
3,5,D


In [3]:
r2

,C1,C3
0,7,E
1,3,F
2,1,J
3,2,L


In [4]:
r1.njoin(r2)

,C1,C2,C3
0,1,A,J
1,2,B,L
2,3,C,F


In [5]:
%load_ext sql


In [6]:
%%sql

postgresql://yasiro01:@localhost/jtest


'Connected: yasiro01@jtest'

### Cartesian Product: SQL


In [7]:
%%sql

select *
from r1, r2;

16 rows affected.


C1,C2,C1_1,C3
1,A,7,E
1,A,3,F
1,A,1,J
1,A,2,L
2,B,7,E
2,B,3,F
2,B,1,J
2,B,2,L
3,C,7,E
3,C,3,F


### Natural Join: SQL

In [8]:
%%sql

select *
from r1 natural join r2;

3 rows affected.


C1,C2,C3
1,A,J
2,B,L
3,C,F


### Natural Join vs Cartesian Product

To return to the cartesian product example, note that it is not 100% equivalent as the cartesian product retains and renames the second copy of column1.


In [9]:
%%sql

select *
from r1, r2
where r1."C1" = r2."C1"

3 rows affected.


C1,C2,C1_1,C3
1,A,1,J
2,B,2,L
3,C,3,F


## Natural Join Example

Now lets look at a more real example. From our city and country tables we have a problem:

* the column name is in both relations, but means different things
* the column population is in both relations but means different things
* the column we would like to join on is the **countrycode** column, but it is called code in the country relation and countrycode in the city relation

We can remedy this in relational algebra by using the rename operator.


In [10]:
city = Relation('/home/faculty/yasiro01/pub/city.csv')
country = Relation('/home/faculty/yasiro01/pub/country.csv')

In [11]:
city.head()

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000.0
1,2,Qandahar,AFG,Qandahar,237500.0
2,3,Herat,AFG,Herat,186800.0
3,4,Mazar-e-Sharif,AFG,Balkh,127800.0
4,5,Amsterdam,NLD,Noord-Holland,731200.0


In [12]:
country.head()

,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,15864000,78.3,371362.0,360478.0,Nederland,Constitutional Monarchy,Beatrix,5.0,NL
2,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.0,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33.0,AN
3,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
4,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,31471000,69.7,49982.0,46966.0,Al-Jazair/Algérie,Republic,Abdelaziz Bouteflika,35.0,DZ


In [13]:
city.rename('name', 'cname').rename('population', 'pop').head()

,id,cname,countrycode,district,pop
0,1,Kabul,AFG,Kabol,1780000.0
1,2,Qandahar,AFG,Qandahar,237500.0
2,3,Herat,AFG,Herat,186800.0
3,4,Mazar-e-Sharif,AFG,Balkh,127800.0
4,5,Amsterdam,NLD,Noord-Holland,731200.0


### Find all cities in Norway

Relational Algebra


In [14]:
country.query("name == 'Norway'")

,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
148,NOR,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO


In [15]:
city.query("countrycode == 'NOR'")

,id,name,countrycode,district,population
2806,2807,Oslo,NOR,Oslo,508726.0
2807,2808,Bergen,NOR,Hordaland,230948.0
2808,2809,Trondheim,NOR,Sør-Trøndelag,150166.0
2809,2810,Stavanger,NOR,Rogaland,108848.0
2810,2811,Bærum,NOR,Akershus,101340.0


In [26]:
city.rename('countrycode', 'code').rename('name', 'city_name').\
njoin(country.rename('population', 'country_pop')).\
query('name == "Norway"')


,id,city_name,code,district,population,name,continent,region,surfacearea,indepyear,country_pop,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
2806,2807,Oslo,NOR,Oslo,508726.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2807,2808,Bergen,NOR,Hordaland,230948.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2808,2809,Trondheim,NOR,Sør-Trøndelag,150166.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2809,2810,Stavanger,NOR,Rogaland,108848.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2810,2811,Bærum,NOR,Akershus,101340.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO


### Find all cities in Norway

Structured Query Language


In [17]:
%sql postgresql://yasiro01:@localhost/world


'Connected: yasiro01@world'

In [18]:
%%sql

select *
from city, country
where country.name = 'Norway'

4079 rows affected.


id,name,countrycode,district,population,code,name_1,continent,region,surfacearea,indepyear,population_1,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
1,Kabul,AFG,Kabol,1780000,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
2,Qandahar,AFG,Qandahar,237500,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
3,Herat,AFG,Herat,186800,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
4,Mazar-e-Sharif,AFG,Balkh,127800,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
5,Amsterdam,NLD,Noord-Holland,731200,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
6,Rotterdam,NLD,Zuid-Holland,593321,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
7,Haag,NLD,Zuid-Holland,440900,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
8,Utrecht,NLD,Utrecht,234323,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
9,Eindhoven,NLD,Noord-Brabant,201843,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
10,Tilburg,NLD,Noord-Brabant,193238,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO


In [19]:
%%sql
select *
from city join country on code = countrycode
where country.name = 'Norway'

5 rows affected.


id,name,countrycode,district,population,code,name_1,continent,region,surfacearea,indepyear,population_1,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
2807,Oslo,NOR,Oslo,508726,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
2808,Bergen,NOR,Hordaland,230948,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
2809,Trondheim,NOR,Sør-Trøndelag,150166,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
2810,Stavanger,NOR,Rogaland,108848,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO
2811,Bærum,NOR,Akershus,101340,NOR,Norway,Europe,Nordic Countries,323877.0,1905,4478500,78.7,145895.00,153370.00,Norge,Constitutional Monarchy,Harald V,2807,NO


## Movie Database

The natural join operator works very well on the movie database as it has two columns with the same name in both the moviecast table and the release_date table.


### Find the names of all of the lead actors in the  movies released in October of 2015 in Norway

In [20]:
moviecast = Relation('/home/faculty/yasiro01/pub/cast.csv',sep=',')
release_date = Relation('/home/faculty/yasiro01/pub/release_dates.csv',sep=',')

In [21]:
moviecast.head()

,title,year,name,type,character,n
0,Star Trek: First Contact,1996,Joey Anaya,actor,Borg,NaN
1,Apache Trail,1942,Donna Reed,actress,Rosalia Martinez,2.0
2,My Favorite Martian,1999,Michael Adler,actor,Scientist,39.0
3,Contact,1997,Jodie Foster,actress,Eleanor Arroway,3.0
4,Caddyshack,1980,Bill Murray,actor,Carl Spackler,5.0


In [22]:
release_date.head()

,title,year,country,date,month,day,dow
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15,6,15,4
1,#AnonOccupy: Bio of a Villain,2015,USA,2015-11-05,11,5,3
2,#Beings,2015,Romania,2015-01-29,1,29,3
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21,1,21,2
4,#Horror,2015,USA,2015-11-20,11,20,4


In [27]:
release_date.\
njoin(moviecast).\
query("country == 'Norway' & n == 1 & year == 2015 & month == 10").\
project(['title', 'name', 'date'])

,title,name,date
1699579,Black Mass,Johnny Depp,2015-10-23
2850250,Crimson Peak,Mia Wasikowska,2015-10-16
5577037,Hotel Transylvania 2,Adam Sandler,2015-10-30
6707245,Klovn Forever,Casper Christensen,2015-10-09
7187906,Legend,Paul (XVIII) Anderson,2015-10-16
8050948,Mia madre,Margherita Buy,2015-10-23
9242941,Pan,Hugh Jackman,2015-10-07
9291999,Paranormal Activity: The Ghost Dimension,Chris J. Murray,2015-10-23
10336246,Rudhramadevi,Anushka Shetty,2015-10-09
10636360,Scouts Guide to the Zombie Apocalypse,Tye Sheridan,2015-10-30


When you run the above query it takes a while, because it is doing a very large join. However we can make it run more quickly by reducing the size of the relations involved in the join by using a query on each.

In [40]:
moviecast.\
query("n == 1").\
njoin(release_date.query("country == 'Norway' & year == 2015 & month == 10")).\
project(['title', 'name', 'date'])

,title,name,date
0,Legend,Paul (XVIII) Anderson,2015-10-16
1,Klovn Forever,Casper Christensen,2015-10-09
2,The Martian,Matt Damon,2015-10-02
3,The Intern,Robert De Niro,2015-10-02
4,Black Mass,Johnny Depp,2015-10-23
5,The Walk (II),Joseph Gordon-Levitt,2015-10-09
6,Pan,Hugh Jackman,2015-10-07
7,Spectre,Solomon Taiwo Justified,2015-10-30
8,Paranormal Activity: The Ghost Dimension,Chris J. Murray,2015-10-23
9,Hotel Transylvania 2,Adam Sandler,2015-10-30


In [23]:
%sql postgresql://yasiro01:@localhost/movies


'Connected: yasiro01@movies'

In [41]:
%%sql

select title, name, date
from moviecast natural join release_date
where month = 10 and year = 2015 and country = 'Norway' and n = 1;


15 rows affected.


title,name,date
Black Mass,Johnny Depp,2015-10-23 00:00:00
Crimson Peak,Mia Wasikowska,2015-10-16 00:00:00
Hotel Transylvania 2,Adam Sandler,2015-10-30 00:00:00
Klovn Forever,Casper Christensen,2015-10-09 00:00:00
Legend,Paul (XVIII) Anderson,2015-10-16 00:00:00
Mia madre,Margherita Buy,2015-10-23 00:00:00
Pan,Hugh Jackman,2015-10-07 00:00:00
Paranormal Activity: The Ghost Dimension,Chris J. Murray,2015-10-23 00:00:00
Rudhramadevi,Anushka Shetty,2015-10-09 00:00:00
Scouts Guide to the Zombie Apocalypse,Tye Sheridan,2015-10-30 00:00:00
